In [1]:
import bs4
import pandas as pd
import requests
import xlsxwriter

In [47]:
#tehcrunch url tags for scrapping
path_urls = r"C:\Users\Ravit\Documents\rnd\horizon_scanning_lab\Scrapers\techcrunch_RCS\url_tags_for_scrapping.csv"

In [49]:
url_df = pd.read_csv(path_urls)

In [50]:
url_df

,domain,url
0,Agriculture,https://techcrunch.com/tag/agriculture-tech/
1,Agriculture,https://techcrunch.com/tag/agtech/
2,Finance,https://techcrunch.com/tag/financial-technology/
3,Energy,https://techcrunch.com/tag/energy-tech/
4,Energy,https://techcrunch.com/tag/renewable-energy/
5,Climate & Geo,https://techcrunch.com/tag/climate-tech/
6,Military,https://techcrunch.com/tag/military-tech/
7,Space,https://techcrunch.com/tag/space-technology/
8,Medicine,https://techcrunch.com/tag/medtech/
9,Medicine,https://techcrunch.com/tag/medical-technology/


In [52]:
response = requests.get(url_df["url"].iloc[0])

soup = bs4.BeautifulSoup(response.text, "html.parser")  

In [59]:
soup.find_all("a", {"class":"post-block__title__link"})[0].find_next(text=True).strip()


'If it’s agtech, it’s climate change: How the crisis is shaping investors’ strategies'

In [60]:
def get_articles_and_hrefs(link):
    
    response = requests.get(link)
    soup = bs4.BeautifulSoup(response.text, "html.parser")  
    
    hrefs_list = []
    title_list = []
    
    link_elements = soup.find_all("a", {"class":"post-block__title__link"})

    for link_el in link_elements:
        hrefs_list.append(link_el['href'])
        title_list.append(link_el.find_next(text=True).strip())

    return hrefs_list, title_list

In [64]:
cols = ["domain", "link", "title"]
out_df = pd.DataFrame([], columns = cols)

for i, row in url_df.iterrows():
    temp_df = pd.DataFrame([], columns = cols)
    temp_df['link'], temp_df['title'] = get_articles_and_hrefs(row['url'])
    temp_df['domain'] = [row["domain"]]*len(temp_df)
    out_df = pd.concat([out_df, temp_df])
    

In [65]:
out_df

,domain,link,title
0,Agriculture,https://techcrunch.com/2022/10/07/if-its-agtec...,"If it’s agtech, it’s climate change: How the c..."
1,Agriculture,https://techcrunch.com/2022/10/05/7-investors-...,7 investors discuss how agtech can solve agric...
2,Agriculture,https://techcrunch.com/2021/12/16/phytoform-cl...,Phytoform plants $5.7M into climate-resistant ...
3,Agriculture,https://techcrunch.com/2021/09/14/agbiome-land...,AgBiome lands $116M for safer crop protection ...
4,Agriculture,https://techcrunch.com/2019/03/07/vcs-have-gro...,VCs have growing appetite for ‘AgriFood’
...,...,...,...
3,Legal,https://techcrunch.com/2020/10/28/priori-serie...,Priori raises $6.3M to help large companies hi...
4,Legal,https://techcrunch.com/2018/08/17/klarity-uses...,Klarity uses AI to strip drudgery from contrac...
5,Legal,https://techcrunch.com/2018/04/12/helpself-use...,HelpSelf uses simple AI to help those in legal...
6,Legal,https://techcrunch.com/2016/08/23/logikcull-ra...,Logikcull raises $10M to let lawyers analyze d...


In [66]:
out_df.to_csv(r"C:\Users\Ravit\Documents\rnd\horizon_scanning_lab\Scrapers\techcrunch_RCS\scrapped_titles\techcrunch_scrapped_article_titles.csv")